# **04.Results**

Summary:


1.   Import data
2.   Define functions
3.   Testing




---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import sys
sys.path.append('/content/drive/My Drive/Università/inforet_prj/')

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
import pickle
import string
import seaborn as sns; sns.set()
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
from nltk import ngrams
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

## **1. Import data**

In [5]:
with open("/content/drive/MyDrive/Università/inforet_prj/df_tfidf_1.pkl", "rb") as f:
    df_tfidf_1 = pickle.load(f)

In [6]:
with open("/content/drive/MyDrive/Università/inforet_prj/df_tfidf_2.pkl", "rb") as f:
    df_tfidf_2 = pickle.load(f)

---

In [7]:
with open("/content/drive/MyDrive/Università/inforet_prj/df_tfidf_50s.pkl", "rb") as f:
    df_tfidf_50s = pickle.load(f)

In [8]:
with open("/content/drive/MyDrive/Università/inforet_prj/df_tfidf_60s.pkl", "rb") as f:
    df_tfidf_60s = pickle.load(f)

In [9]:
with open("/content/drive/MyDrive/Università/inforet_prj/df_tfidf_70s.pkl", "rb") as f:
    df_tfidf_70s = pickle.load(f)

In [10]:
with open("/content/drive/MyDrive/Università/inforet_prj/df_tfidf_80s.pkl", "rb") as f:
    df_tfidf_80s = pickle.load(f)

In [11]:
with open("/content/drive/MyDrive/Università/inforet_prj/df_tfidf_90s.pkl", "rb") as f:
    df_tfidf_90s = pickle.load(f)

In [12]:
with open("/content/drive/MyDrive/Università/inforet_prj/df_tfidf_00s.pkl", "rb") as f:
    df_tfidf_00s = pickle.load(f)

---

In [13]:
with open("/content/drive/MyDrive/Università/inforet_prj/models.pkl", "rb") as f:
    models = pickle.load(f)

In [14]:
with open("/content/drive/MyDrive/Università/inforet_prj/models_time.pkl", "rb") as f:
    models_time = pickle.load(f)

---

In [15]:
with open("/content/drive/MyDrive/Università/inforet_prj/coocc_op.pkl", "rb") as f:
    cooc_op = pickle.load(f)

In [16]:
with open("/content/drive/MyDrive/Università/inforet_prj/coocc_1_pmi.pkl", "rb") as f:
    coocc_1_pmi = pickle.load(f)

In [17]:
with open("/content/drive/MyDrive/Università/inforet_prj/coocc_2_pmi.pkl", "rb") as f:
    coocc_2_pmi = pickle.load(f)

---

In [18]:
with open("/content/drive/MyDrive/Università/inforet_prj/coocc_bigr_1_pmi.pkl", "rb") as f:
    coocc_bigr_1_pmi = pickle.load(f)

In [19]:
with open("/content/drive/MyDrive/Università/inforet_prj/coocc_bigr_2_pmi.pkl", "rb") as f:
    coocc_bigr_2_pmi = pickle.load(f)

---

In [20]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_op_matrix.pkl", "rb") as f:
    pmi_op_matrix = pickle.load(f)

In [21]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_matrix_1_pmi.pkl", "rb") as f:
    pmi_matrix_1_pmi = pickle.load(f)

In [22]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_matrix_2_pmi.pkl", "rb") as f:
    pmi_matrix_2_pmi = pickle.load(f)

---

In [23]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_1_pmi_bigr_matrix.pkl", "rb") as f:
    pmi_1_pmi_bigr_matrix = pickle.load(f)

In [24]:
with open("/content/drive/MyDrive/Università/inforet_prj/pmi_2_pmi_bigr_matrix.pkl", "rb") as f:
    pmi_2_pmi_bigr_matrix = pickle.load(f)

---

## **2. Define functions**

In [25]:
def find_tfidf(df, term, value=False):
    row = df.loc[df.word == term]

    if row.shape[0] == 0:
        return term +' not in vocabulary'
    
    if value:
        return row.index.values[0], term, row.value.values[0]
    
    return row.index[0], term

In [26]:
def most_similar_list_schedule(model_, narcos, word, n, value=False):
    lista = []
    for sched, model in model_.items():
        if sched == narcos:
            for x in model.wv.most_similar(positive=word)[:n]:
                if value:
                    lista.append([x[0],round(x[1], 2)])
                    #print(sched, x[0],round(x[1], 2))
                else:
                    lista.append(x[0])
                    #print(sched, x[0])
    return lista

In [27]:
def most_similar_list_time(model_, time, word, n, value=False):
    lista = []
    for year, model in model_.items():
        if year == time:
            for x in model.wv.most_similar(positive=word)[:n]:
                if value:
                    lista.append([x[0],round(x[1], 2)])
                    #print(year, x[0],round(x[1], 2))
                else:
                    lista.append([x[0]])
                    #print(year, x[0])
    return lista

In [28]:
def similarity_2_words_time(model_, word1, word2):
    print('(' , word1 , '-' , word2, ')')
    for year, model in model_.items():
        try:
            print(year, round(model.wv.similarity( word1, word2 ), 3) )
        except:
            print(year, 'No data to display')
            #pass

In [29]:
def get_co_pmi_value(word1, word2, matrix):
    return matrix.loc[matrix.index == word1, [word2]]

In [30]:
def pmi(df, positive=True):
    col_totals = df.sum(axis=0)
    total = col_totals.sum()
    row_totals = df.sum(axis=1)
    expected = np.outer(row_totals, col_totals) / total
    df_pmi = df / expected
    # Silence distracting warnings about log(0):
    with np.errstate(divide='ignore'):
        df_pmi = np.log(df_pmi)
    df_pmi[np.isinf(df_pmi)] = 0.0  # log(0) = 0
    if positive:
        df_pmi[df_pmi < 0] = 0.0
    return df_pmi

## **3. Testing**

In [31]:
find_tfidf(df_tfidf_1, 'drug', True)

(43, 'drug', 0.04690238904079548)

In [32]:
most_similar_list_schedule(models, 'narco_2', 'drug', 10, True)

[['narcotic', 0.8],
 ['cocaine', 0.72],
 ['buying', 0.5],
 ['amphetamine', 0.49],
 ['occasional', 0.48],
 ['consummate', 0.46],
 ['ongoing', 0.46],
 ['dope', 0.46],
 ['profit', 0.46],
 ['substance', 0.44]]

In [33]:
most_similar_list_time(models_time, '60s', 'drug', 10, True)

[['dispense', 0.83],
 ['dispensing', 0.8],
 ['selling', 0.74],
 ['possess', 0.74],
 ['transport', 0.73],
 ['unlawfully', 0.72],
 ['trade', 0.72],
 ['profess', 0.71],
 ['user', 0.7],
 ['compound', 0.7]]

In [34]:
similarity_2_words_time(models_time, 'drug','narcotic')

( drug - narcotic )
50s 0.973
60s 0.583
70s 0.486
80s 0.649
90s 0.803
00s 0.756


In [38]:
get_co_pmi_value('drug', 'crime', cooc_op)

,crime
drug,75


In [35]:
get_co_pmi_value('drug', 'crime', pmi_op_matrix)

,crime
drug,0.195879


In [36]:
get_co_pmi_value('drug', 'crime', pmi_matrix_1_pmi)

,crime
drug,0.065747


In [37]:
get_co_pmi_value('drug', 'crime', pmi_matrix_2_pmi)

,crime
drug,0.07536
